Author: Carlos Roberto de Melo

Date: 05/27/2020

Obj: Perform the modeling of spectrum based on pPXF method by Cappellari,and obtein the kinematics./Realizar o modelamento de espectros baeado no código pPXF de Cappellari, e obter a cinemática.

In [1]:
import glob
from os import path

from astropy.io import fits
from scipy import ndimage
import numpy as np

import ppxf as ppxf_package
from ppxf.ppxf import ppxf
import ppxf.ppxf_util as util

import matplotlib.pyplot as plt

Adaptation of the Ppxf code to apply different spectra.
**x** and **y** are positions in px, which serves as counter and also index to open the spectrum files, which were saved in the format **y_px X x_px.fits**.

As discussed in the README, the .fits files containing the spectrum have three extensions. Note that pyton, by default, starts a count of zero.

It should be viewed as units of wavelength and flux. By default, pPXF works with wavelength in Angstrons, as we are using the MILES star library. In our case, as we normalize the spectrum, as units of intensity should not cause problems.



Adapatação do código Ppxf para se aplicar a diversos espectros.
x e y são as posições em px, de modo que servem de contador e também de index para abrir os arquivos de espectro, que foram salvos no formato **y_px X x_px.fits**.

Como discutido no README, os arquivos .fits que contém os espectros possuem três extensões. Note que o pyton, por padrão, começa a contagem do zero.

Deve-se antentar as unidades de comprimento de onda e fluxo. Por padrão, o ppxf trabalha com comprimento de onda em Angstrons pois estamos utilizando a biblioteca estelar do MILES. No nosso caso como vamos normalizar o espectro, as unidades de intensidade não devem gerar problemas.



In [2]:
ppxf_dir = '/home/carlos/Desktop/ppxf' #Directory of stellar library/Diretória da biblioteca estelar
cont = 0
for y in range(20):
    for x in range(20):
        chi2_dof = 1
    #READING THE SPECTRUM/LENDO UM ESPECTRO
        file = ('/home/carlos/Documents/GitHub/Master-Degree/ESO325/Dynamical Modelling/pPXF/Espectros/%dx%d.fits'%(y,x)) #Path and name of the spectrum/Caminho e nome do arquivo do espectro
        hdu = fits.open(file) #Opening the .fits file/Abrindo o arquivo.fits
        gal_lin = hdu[1].data #Read the data/Lendo os dados
        h1 = hdu[1].header    #Read the header/Lendo o header dos dados
        
    #READING THE VARIANCE/LENDO A VARIÂNCIA DO ESPECTRO
        var = hdu[2].data     #Read the variance/Lendo a variância
        h_var = hdu[2].header #Read the variance header/Lendo o header da variância
        
        
        
    #LOG-REBIN THE SPECTRUM AND VARIANCE/LOGARITMIZANDO O ESPECTRO E A VARIÂNCIA
        #SPECTRUM/ESPECTRO
        FWHM_gal = 2.71  #FWHM of telescope in Angstrons/FWHM do telescópio em Angstrons
        lambRange1 = h1['CRVAL1'] + np.array([0.,h1['CDELT1']*(h1['NAXIS1']-1)]) #Generating the wavelengths/Está linha gera os comprimentos de onda
                                #pois a variável gal_lin armazena em um array apenas as intensidades, mas não os comprimentos de onda
        
        
        #De-redshift
        z = 0.035 # Initial estimate of the galaxy redshift
        lambRange1 = lambRange1/(1+z) # Compute approximate restframe wavelength range
        FWHM_gal = FWHM_gal/(1+z)   # Adjust resolution in Angstrom
        #Depois do passo acima, a galáxia passa a estar no restframe, logo é preeciso setar
        z = 0.
        
        
        galaxy, logLam1, velscale = util.log_rebin(lambRange1, gal_lin) #Log-rebin the spectrum/Logaritimizando o espectro da galáxia
        median_galaxy = np.median(galaxy) #Mean of spectrum for normalization/Média do espectro, pois vamos normalizá-lo pela média
        galaxy = galaxy/median_galaxy        #Normalizing the spectrum/Normalizando o espectro pela média
        lam = np.exp(logLam1)                #Recovering the wavelength in Ansgstrons/Recuperando o comprimento de onda em unidades do SI
        
        #VARIANCE/VARIÂNCIA
        lamRange_var = h_var['CRVAL1'] + np.array([0.,h_var['CDELT1']*(h_var['NAXIS1']-1)])
        erro = np.sqrt(chi2_dof*var) #1-sigma error from variance/1-sigma erro da variância
        erro, loglam_var, velscale_var = util.log_rebin(lamRange_var, erro)
        noise = erro/median_galaxy         #Normalizing the error/Normalizando o erro        
        
    #READING ONE TEMPLATE FOR DETERMINE THE LENGHT/LENDO UM TEMPLATE PARA SABER O TAMANHO DO VETOR FINAL
        #The ideia is make a empty array to after complete with the informations about the templates of the stellar libary/
                    #A ideia é crirar um array vazio para que mais tarde seja preenchido com as informações de todos os templetes.
        vazdekis = glob.glob(ppxf_dir + '/miles_models/Mun1.30*.fits') 
        FWHM_tem = 2.51  #FWHM of templates in Angstrons/FWHM do templete em Angstrons
        velscale_ratio = 2
        hdu = fits.open(vazdekis[0])
        ssp = hdu[0].data
        h2 = hdu[0].header
        
        #Log_rebin TEMPLATE/LOGARITMIZANDO O TEMPLATE
        lamRange2 = h2['CRVAL1'] + np.array([0., h2['CDELT1']*(h2['NAXIS1'] - 1)])
        sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
        templates = np.empty((sspNew.size, len(vazdekis)))  #Array with necessary size/Array com o tamanho necessário
    
    #FINAL FWHM FOR GALAXY AND TEMPLATE CONVOLUTION/CALCULANDO A FWHM FINAL PARA REALIZAR A CONVOLUÇÃO DO TEMPLATE E GALÁXIA
        FWHM_dif = np.sqrt(FWHM_gal**2 - FWHM_tem**2)
        sigma = FWHM_dif/2.355/h2['CDELT1'] #New sigma PSF/Novo sigma da PSF.
    
    #READING ALL TEMPLATES/LENDO TODOS OS TEMPLATES E JÁ LOGARITMIZANDO
        for j, file in enumerate(vazdekis):
            hdu = fits.open(file)
            ssp = hdu[0].data
            ssp = ndimage.gaussian_filter1d(ssp, sigma) #Convolution/Realizando a convolução
            sspNew, logLam2, velscale_temp = util.log_rebin(lamRange2, ssp, velscale=velscale/velscale_ratio)
            templates[:, j] = sspNew/np.median(sspNew)
    
    #ADICIONANDO UM SHIFT PARA COINCIDIR OS COMPRIMENTOS DE ONDA DO ESPECTRO E DO TEMPLATE
        #Artificial shift for the galaxy and templates wavelenght may coincide.    
        #Nem sempre os comprimentos de onda do templete e do espectro coincidem, por esse motivo é dado um 
        #shift artificial ao templete, mas que após o ajuste é retirado.
        c = 299792.458 #Speed of light/Velocidade da Luz
        dv = (np.mean(logLam2[:velscale_ratio]) - logLam1[0])*c #Shift
        goodPixels = util.determine_goodpixels(logLam1, lamRange2, z)  #Mask bad emission lines/Máscaras para linhas de emissão
        vel = c*np.log(1 + z)
        start = [vel, 200.] #Inicial guess/Chute inicial
    
    #START OF FIT/INÍCIO DO AJUSTE
       #For each spectrum the modeling is done. After the best solution is printed 
       #(velocity, velocity dispersion, another moments). Then, a table is write with the information about the
       #pixel position, velocity, velocity dispersion, chi²/DOF and 1-sigma error. The table is saved with name
       #'Output_Ppxf.txt'.  
        
       #Para cada espectro ele realiza o ajuste e printa abaixo o nome do px em que foi realizado o ajuste,
       #bem como as infomações do melhor ajuste (qui², Velocidade, Dispersão, Demais momentos). Então cria uma
       #tabela com as informações necessárias e a salva com o nome 'Output_Ppxf.txt'.
        pp = ppxf(templates, galaxy, noise, velscale, start,
                lam = lam, plot=False, moments=2,
                  degree=6,vsyst=dv, goodpixels=goodPixels, velscale_ratio=velscale_ratio)
        #For more informations about the method, visit the paper of Cappellari avaible in README
        #Para mais informações sobre o método, visite o paper de Cappellari disponível no README
        
        #Plot and save the best model/Plota e salva o melhor modelo
        pp.plot() #Plot the fit/Plota o fit
        plt.savefig('/home/carlos/Documents/GitHub/Master-Degree/ESO325/Dynamical Modelling/pPXF/Test/%dx%d.png'%(y,x),format='png', dpi=1200)
        plt.clf()
        cont = cont+1
        print(y,x)
        print('Vel :%f.\nDisp:%f.'%(pp.sol[0],pp.sol[1]))
        print('Best-fitting redshift z:', (0.035 + 1)*(1 + pp.sol[0]/c) - 1)
        error = pp.error*np.sqrt(pp.chi2) #Formal Error. dV = error[0], dsigma = error[1]
    #ESCREVENDO UMA TABELA COM OS DADOS DE Y_PX, X_PX,  VELOCIDADE(KM/S) E DISPERSÃO(KM/S)
        newrow = [np.int(y), np.int(x), pp.sol[0], pp.sol[1], pp.chi2, error[0], error[1]]
        if y == 0 and x == 0:
            table = np.array([np.int(y),np.int(x),pp.sol[0], pp.sol[1],pp.chi2, error[0], error[1]])
        else:
            table = np.vstack([table,newrow])
        print('\n####################\n')

 Best Fit:       Vel     sigma
 comp.  0:      -343       304
chi2/DOF: 1.714
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
0 0
Vel :-343.291842.
Disp:303.612838.
Best-fitting redshift z: 0.033814823232272584

####################

 Best Fit:       Vel     sigma
 comp.  0:      -344       244
chi2/DOF: 2.098
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
0 1
Vel :-344.408458.
Disp:243.743878.
Best-fitting redshift z: 0.03381096824000229

####################

 Best Fit:       Vel     sigma
 comp.  0:      -327       392
chi2/DOF: 1.721
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
0 2
Vel :-327.498765.
Disp:392.317689.
Best-fitting redshift z: 0.03386934706738276

####################

 Best Fit:       Vel     sigma
 comp.  0:      -271       321
chi2/DOF: 1.512
method = capfit; Jac 

1 7
Vel :-292.671555.
Disp:290.393047.
Best-fitting redshift z: 0.033989584123691685

####################

 Best Fit:       Vel     sigma
 comp.  0:      -293       324
chi2/DOF: 1.403
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
1 8
Vel :-292.810387.
Disp:323.525236.
Best-fitting redshift z: 0.03398910482137918

####################

 Best Fit:       Vel     sigma
 comp.  0:      -335       299
chi2/DOF: 1.451
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
1 9
Vel :-334.925868.
Disp:298.796093.
Best-fitting redshift z: 0.033843705821821723

####################

 Best Fit:       Vel     sigma
 comp.  0:      -325       301
chi2/DOF: 1.487
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  5 / 150
1 10
Vel :-325.438423.
Disp:301.296070.
Best-fitting redshift z: 0.03387646016686596

############

 Best Fit:       Vel     sigma
 comp.  0:      -331       330
chi2/DOF: 1.598
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
2 15
Vel :-330.698427.
Disp:329.802876.
Best-fitting redshift z: 0.033858300590617896

####################

 Best Fit:       Vel     sigma
 comp.  0:      -303       310
chi2/DOF: 1.433
method = capfit; Jac calls: 5; Func calls: 17; Status: 4
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
2 16
Vel :-303.076676.
Disp:309.892750.
Best-fitting redshift z: 0.03395366160247737

####################

 Best Fit:       Vel     sigma
 comp.  0:      -263       308
chi2/DOF: 1.466
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
2 17
Vel :-263.345107.
Disp:308.126508.
Best-fitting redshift z: 0.0340908304115346

####################

 Best Fit:       Vel     sigma
 comp.  0:      -287       297
chi2/DOF: 1.422
method = capfit; Ja

4 2
Vel :-326.995271.
Disp:270.773595.
Best-fitting redshift z: 0.03387108532270533

####################

 Best Fit:       Vel     sigma
 comp.  0:      -353       324
chi2/DOF: 1.434
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
4 3
Vel :-352.580688.
Disp:323.650215.
Best-fitting redshift z: 0.03378275452787083

####################

 Best Fit:       Vel     sigma
 comp.  0:      -305       310
chi2/DOF: 1.35
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
4 4
Vel :-304.868573.
Disp:310.150079.
Best-fitting redshift z: 0.03394747528115749

####################

 Best Fit:       Vel     sigma
 comp.  0:      -296       301
chi2/DOF: 1.413
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
4 5
Vel :-295.948788.
Disp:300.852259.
Best-fitting redshift z: 0.03397826984107599

################

 Best Fit:       Vel     sigma
 comp.  0:      -323       321
chi2/DOF: 1.237
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
5 10
Vel :-322.544314.
Disp:321.310804.
Best-fitting redshift z: 0.03388645175730942

####################

 Best Fit:       Vel     sigma
 comp.  0:      -319       341
chi2/DOF: 1.209
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
5 11
Vel :-319.114480.
Disp:340.700061.
Best-fitting redshift z: 0.033898292876899605

####################

 Best Fit:       Vel     sigma
 comp.  0:      -322       349
chi2/DOF: 1.403
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
5 12
Vel :-322.146400.
Disp:348.718669.
Best-fitting redshift z: 0.03388782550900915

####################

 Best Fit:       Vel     sigma
 comp.  0:      -311       314
chi2/DOF: 1.11
method = capfit; Ja

...

15 2
Vel :-284.450242.
Disp:294.211721.
Best-fitting redshift z: 0.03401796728777251

####################

 Best Fit:       Vel     sigma
 comp.  0:      -286       309
chi2/DOF: 1.304
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
15 3
Vel :-286.268265.
Disp:308.574900.
Best-fitting redshift z: 0.03401169076707933

####################

 Best Fit:       Vel     sigma
 comp.  0:      -302       312
chi2/DOF: 1.358
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
15 4
Vel :-301.997742.
Disp:312.367379.
Best-fitting redshift z: 0.03395738650291946

####################

 Best Fit:       Vel     sigma
 comp.  0:      -315       321
chi2/DOF: 1.232
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
15 5
Vel :-315.314797.
Disp:321.246083.
Best-fitting redshift z: 0.03391141085787752

###########

 Best Fit:       Vel     sigma
 comp.  0:      -289       329
chi2/DOF: 1.394
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
16 10
Vel :-289.010092.
Disp:328.593059.
Best-fitting redshift z: 0.03400222491378746

####################

 Best Fit:       Vel     sigma
 comp.  0:      -313       304
chi2/DOF: 1.381
method = capfit; Jac calls: 6; Func calls: 20; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
16 11
Vel :-312.885943.
Disp:304.047372.
Best-fitting redshift z: 0.033919796204128216

####################

 Best Fit:       Vel     sigma
 comp.  0:      -303       327
chi2/DOF: 1.158
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  4 / 150
16 12
Vel :-302.817935.
Disp:327.193481.
Best-fitting redshift z: 0.03395455488003862

####################

 Best Fit:       Vel     sigma
 comp.  0:      -321       299
chi2/DOF: 1.279
method = capfit

17 17
Vel :-330.434348.
Disp:274.766328.
Best-fitting redshift z: 0.03385921229363853

####################

 Best Fit:       Vel     sigma
 comp.  0:      -275       334
chi2/DOF: 1.591
method = capfit; Jac calls: 4; Func calls: 14; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
17 18
Vel :-275.335964.
Disp:333.909784.
Best-fitting redshift z: 0.03404943331516863

####################

 Best Fit:       Vel     sigma
 comp.  0:      -321       285
chi2/DOF: 1.454
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
17 19
Vel :-321.054905.
Disp:284.681565.
Best-fitting redshift z: 0.0338915937747748

####################

 Best Fit:       Vel     sigma
 comp.  0:      -304       292
chi2/DOF: 1.443
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
18 0
Vel :-303.525491.
Disp:292.237793.
Best-fitting redshift z: 0.033952112121389266

########

 Best Fit:       Vel     sigma
 comp.  0:      -373       314
chi2/DOF: 1.56
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
19 5
Vel :-373.194629.
Disp:314.122430.
Best-fitting redshift z: 0.03371158719604872

####################

 Best Fit:       Vel     sigma
 comp.  0:      -334       264
chi2/DOF: 1.45
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  2 / 150
19 6
Vel :-334.019284.
Disp:264.201250.
Best-fitting redshift z: 0.03384683570364255

####################

 Best Fit:       Vel     sigma
 comp.  0:      -342       282
chi2/DOF: 1.409
method = capfit; Jac calls: 5; Func calls: 17; Status: 2
linear_method = nnls; Nonzero Templates (>0.1%):  3 / 150
19 7
Vel :-342.106205.
Disp:281.789633.
Best-fitting redshift z: 0.03381891651218294

####################

 Best Fit:       Vel     sigma
 comp.  0:      -265       309
chi2/DOF: 1.644
method = capfit; Jac 

<Figure size 432x288 with 0 Axes>

In [3]:
np.savetxt('Test.txt',table, header= 'y_px x_px vel(km/s) disp(km/s) chi2/DOF dV dsigma', fmt='%i %i %.18e %.18e %.18e %.18e %.18e',delimiter=' ') 